# Prompt + LLM

#### 1. 필수 라이브러리 설치

In [8]:
!pip install --upgrade --quiet langchain langchain-core langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 11.8 MB/s eta 0:00:00


#### 2. OpenAI API Key 설정

In [2]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-**********************************************"

## PromptTemplate + LLM

Prompt Template과 LLM 결합

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#Prompt 생성
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

#model 생성
model = ChatOpenAI()

#LCEL 구조로 Chain 생성
chain = prompt | model

#Prompt Input 변수 값을 부여하여 실행
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Attaching Stop Sequences

Output 값에 특정 변수가 포함되면 정지되도록 설정

In [4]:
# Model이 Output을 생성할 때, \n 기호가 포함되면 실행을 중지함.
chain = prompt | model.bind(stop=["\n"])

chain.invoke({"foo": "bears"})

AIMessage(content='Why did the bear dissolve in water?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Attaching Function Call information

모델이 특정 함수를 실행하도록 설정
- function_call : 함수 내에서 사용할 특정 함수 이름으로 선택
- functions : 사용할 함수 리스트 지정

In [5]:
#joke라는 이름이 포함된 함수를 json Type으로 함수 목록에 저장 
# properties 에서 답변의 구조를 설정
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

#특정 함수를 모델에 적용함
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

chain.invoke({"foo": "bears"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"Why don\'t bears like fast food?","punchline":"Because they can\'t catch it!"}', 'name': 'joke'}}, response_metadata={'finish_reason': 'stop', 'logprobs': None})

## PromptTemplate + LLM + OutputParser

OutputParser : 지정한 템플릿에 따라 Output 결과를 반환 

In [ ]:
from langchain_core.output_parsers import StrOutputParser

#StrOutputParser : String Type으로 결과값 반환
chain = prompt | model | StrOutputParser()

chain.invoke({"foo": "bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"

### Functions Output Parser

함수에 지정된 결과값 형태를 OutputParser의 데이터 타입으로 반환 

In [9]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

#joke 함수의 output properties 항목에 따라서 함수를 실행
#Json 데이터 타입으로 joke 함수의 결과값을 맵핑하여 반환
chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

chain.invoke({"foo": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

In [10]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

#joke 함수의 output properties 항목에 따라서 함수를 실행 
#Json 형태의 결과값에서 setup 이라는 key값의 데이터만을 결과로 반환
chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke({"foo": "bears"})

"Why don't bears like fast food?"

## Simplifying input

Input 값을 {input_variable_name : input_value} 가 아닌, input_value만 입력하여 출력할 수 있도록 설정

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

#"foo"라는 입력변수명을 RunnableParallel에 입력값으로 사용할 수 있도록 하는 모듈을 불러옴
map_ = RunnableParallel(foo=RunnablePassthrough())

#Parallel을 포함하여 LCEL 구조로 구현
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears wear shoes?"

In [12]:
#"foo" 변수 값을 Pass하여 받도록 지정
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears like fast food?"